## 사용자 행동 인식 데이터 (다중분류)
`04_classification/homework_har_251203.ipynb`에 작성하세요. # 155

https://www.kaggle.com/datasets/uciml/human-activity-recognition-with-smartphones 


## 요구사항
- 이 데이터에 적합한 최고의 성능을 가진 분류 모델을 찾아보세요.
- 최적의 하이퍼 파라미터설정이 필요합니다.

아래는 모델별 성능비교표 예시입니다.

모델명 | Accuracy | Precision | Recall | F1-score | AUC | 기타 메모
-- | -- | -- | -- | -- | -- | --
Logistic Regression | 0.89 | 0.88 | 0.87 | 0.87 | 0.91 | 베이스라인 모델
Random Forest | 0.92 | 0.91 | 0.90 | 0.91 | 0.95 | 파라미터 기본값
XGBoost | 0.94 | 0.93 | 0.92 | 0.93 | 0.96 | EarlyStopping 적용
SVM (RBF) | 0.90 | 0.89 | 0.88 | 0.88 | 0.93 | 표준화 필수
KNN (k=5) | 0.85 | 0.84 | 0.83 | 0.83 | 0.87 | 거리기반 특성 영향

> 최고 점수(정확도)를 commit message에 작성해주세요~ 😄 


In [ ]:
!pip install scikit-learn xgboost

In [ ]:
# ============================================
# Human Activity Recognition 분류 실습
# ============================================

# 1. 라이브러리 불러오기
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelBinarizer

# 2. 데이터 로드
train = pd.read_csv("data/train.csv")  # Kaggle에서 train.csv 다운로드 필요
test = pd.read_csv("data/test.csv")    # test.csv 다운로드 필요

# 3. 데이터 확인
print(train.head())
print(train['Activity'].value_counts())

# 4. 특성과 레이블 분리
X_train = train.drop('Activity', axis=1)
y_train = train['Activity']
X_test = test.drop('Activity', axis=1)
y_test = test['Activity']

# 5. 표준화 (SVM, KNN 필요)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 6. 모델 정의
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'),
    "SVM (RBF)": SVC(probability=True),
    "KNN (k=5)": KNeighborsClassifier()
}

# 7. 하이퍼파라미터 설정 (예시)
param_grids = {
    "Logistic Regression": {
        'C':[0.1,1,10]
    },
    "Random Forest": {
        'n_estimators':[100,200],
        'max_depth':[10,20,None],
        'min_samples_split':[2,5]
    },
    "XGBoost": {
        'n_estimators':[100,200],
        'max_depth':[3,6,10],
        'learning_rate':[0.01,0.1,0.2],
        'subsample':[0.8,1]
    },
    "SVM (RBF)": {
        'C':[0.1,1,10],
        'gamma':['scale','auto']
    },
    "KNN (k=5)": {
        'n_neighbors':[3,5,7],
        'weights':['uniform','distance']
    }
}

# 8. 성능 비교표 초기화
results = []

# 9. 모델 학습 및 평가
for name, model in models.items():
    print(f"===== {name} =====")
    
    # SVM, KNN은 스케일링 데이터 사용, 나머지는 원본 사용
    if name in ["SVM (RBF)", "KNN (k=5)"]:
        X_tr, X_te = X_train_scaled, X_test_scaled
    else:
        X_tr, X_te = X_train, X_test
    
    # GridSearchCV를 통한 최적 하이퍼파라미터 탐색
    grid = GridSearchCV(model, param_grids[name], cv=3, scoring='accuracy', n_jobs=-1)
    grid.fit(X_tr, y_train)
    
    best_model = grid.best_estimator_
    y_pred = best_model.predict(X_te)
    
    # 다중 클래스 AUC 계산
    lb = LabelBinarizer()
    y_test_bin = lb.fit_transform(y_test)
    y_pred_prob = best_model.predict_proba(X_te)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    auc = roc_auc_score(y_test_bin, y_pred_prob, average='weighted', multi_class='ovr')
    
    results.append({
        "모델명": name,
        "Accuracy": round(accuracy,3),
        "Precision": round(precision,3),
        "Recall": round(recall,3),
        "F1-score": round(f1,3),
        "AUC": round(auc,3),
        "기타 메모": f"Best Params: {grid.best_params_}"
    })

# 10. 결과 출력
results_df = pd.DataFrame(results)
print(results_df)
